In [1]:
import os

In [3]:
!pip install openai tiktoken chromadb langchain langchain-community GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.4 MB/s eta

In [4]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [5]:
%pwd

'/content'

In [6]:
!mkdir test-repo

In [7]:
repo_path = "test_repo/"
Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation",to_path=repo_path)

<git.repo.base.Repo '/content/test_repo/.git'>

In [24]:
!ls

sample_data  test_repo


Clone Github repositories

In [31]:
repo_path="test_repo/"
loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                       glob='**/*',
                                       suffixes = [".py"],
                                       parser=LanguageParser(language=Language.PYTHON,parser_threshold=500)
                                       )

In [32]:
documents = loader.load()

In [33]:
documents

[Document(metadata={'source': 'test_repo/src/mlProject/__init__.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")'),
 Document(metadata={'source': 'test_repo/src/mlProject/components/data_validation.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nfrom mlProject import logger\nimport pandas as pd\nfrom mlProject.entity.config_entity import DataValidationConfig\n                                    \n\n\n\nclass DataValiadtion:\n    def __init__(self, config: DataValidationConfig):\n        self.

In [34]:
print(documents)

[Document(metadata={'source': 'test_repo/src/mlProject/__init__.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")'), Document(metadata={'source': 'test_repo/src/mlProject/components/data_validation.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nfrom mlProject import logger\nimport pandas as pd\nfrom mlProject.entity.config_entity import DataValidationConfig\n                                    \n\n\n\nclass DataValiadtion:\n    def __init__(self, config: DataValidationConfig):\n        self.c

Chunkings

In [35]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=2000, chunk_overlap=200)

In [36]:
text = documents_splitter.split_documents(documents)

In [37]:
len(text)

19

Embedding Model

In [40]:
from google.colab import userdata
open_api_key = userdata.get('OPEN_API_KEY')

In [41]:
os.environ["OPENAI_API_KEY"] = open_api_key

In [45]:
embeddings = OpenAIEmbeddings(disallowed_special=())

Knowledge base(vector database)

In [46]:
vectordb = Chroma.from_documents(text, embedding=embeddings, persist_directory="./data")
vectordb.persist()

LLM Wrapper

In [47]:
llm = ChatOpenAI()

In [48]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history",return_messages=True)

In [52]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k":3}), memory=memory)

Q&A

In [58]:
# question = "What is data ingestion?"
question ="What is ModelTrianer class?"



In [59]:
result = qa(question)

In [60]:
result

{'question': 'What is ModelTrianer class?',
 'chat_history': [SystemMessage(content='The human asks what data ingestion is. The AI explains that data ingestion is the process of collecting, importing, and processing data for storage or analysis, specifically in the context of downloading and extracting data files for machine learning training. In the context of machine learning training, data ingestion involves configuring setup, data ingestion, data extraction, data preparation, data storage, and training pipeline. The `DataIngestionTrainingPipeline` class handles the data ingestion process by downloading and extracting data files for training a machine learning model.')],
 'answer': 'The `ModelTrainer` class is part of the ML project components and is responsible for training a machine learning model. It is initialized with a configuration object that contains parameters required for training the model. The class has a method `train()` that executes the training process.'}